<a href="https://colab.research.google.com/github/FedericoRaschiatore0123/GATr_Deep_Learning_Project/blob/main/GATr_deep_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
!git clone https://github.com/FedericoRaschiatore0123/GATr_Deep_Learning_Project.git

Cloning into 'GATr_Deep_Learning_Project'...
remote: Enumerating objects: 4122, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4122 (delta 0), reused 0 (delta 0), pack-reused 4119
Receiving objects: 100% (4122/4122), 1.68 GiB | 21.30 MiB/s, done.
Resolving deltas: 100% (71/71), done.
Updating files: 100% (4003/4003), done.


In [2]:
#librerie
!pip install h5py
!pip install trimesh
!pip install torch_geometric

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch_geometric.data
from torch_geometric.data import Data
from torch.utils.data import random_split, ConcatDataset, Subset

import h5py
import os
import trimesh
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.8/693.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00


#Global Variables

In [12]:
single_path ='/content/GATr_Deep_Learning_Project/single/'
bifurcating_path = '/content/GATr_Deep_Learning_Project/bifurcating/'

BATCH_SIZE = 1

#Get device

In [4]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

#Dataset and Dataloader

In [5]:
print(len(os.listdir(single_path)))
print(len(os.listdir(bifurcating_path)))

2000
1999


In [13]:
class Preprocesing():
  def __init__(self, path_single, path_bifurcating):

    self.single_data_path = path_single
    self.bifurcating_data_path = path_bifurcating

  def build_datasets(self):

    single_dataset = self.load_data(self.single_data_path,  bifurcating=False)
    bifurcating_dataset = self.load_data(self.bifurcating_data_path,  bifurcating=True)

    combined_dataset = self.shuffle_and_combine_datasets(single_dataset, bifurcating_dataset)

    return self.split_dataset(combined_dataset)


  def load_data(self, path, bifurcating=False):
    data = []
    for file in os.listdir(path):
      with h5py.File(path + file, 'r') as input_file:
        data.append([
              torch.from_numpy(input_file['wss'][()]),
              torch.from_numpy(input_file['pos'][()]),
              torch.from_numpy(input_file['pressure'][()]),
              torch.from_numpy(input_file['face'][()].T).long(),
              torch.from_numpy(input_file['inlet_idcs'][()])
          ])

    label = torch.ones(len(data)) if bifurcating else torch.zeros(len(data))

    return list(zip(data, label))


  def shuffle_and_combine_datasets(self, dataset1, dataset2):

      combined_dataset = ConcatDataset([dataset1, dataset2])
      indices = torch.randperm(len(combined_dataset))
      shuffled_dataset = Subset(combined_dataset, indices)

      return shuffled_dataset

  def split_dataset(self, dataset, train_ratio=0.7, validation_ratio=0.2, test_ratio=0.1):

      total_size = len(dataset)
      train_size = int(total_size * train_ratio)
      validation_size = int(total_size * validation_ratio)
      test_size = total_size - train_size - validation_size

      train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

      return train_dataset, validation_dataset, test_dataset


dataset = Preprocesing(single_path, bifurcating_path)
train_dataset, validation_dataset, test_dataset = dataset.build_datasets()

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(train_loader)
print(validation_loader)
print(test_loader)

In [14]:
print(next(iter(train_loader)))

[[tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]), tensor([[[-0.1861,  0.0383,  0.0361],
         [-0.1888,  0.0193,  0.0367],
         [-0.1883,  0.0192,  0.0366],
         ...,
         [ 0.0039,  0.1701, -0.0017],
         [ 1.6438,  0.0102,  1.5724],
         [-0.0311, -0.1198,  0.0061]]]), tensor([[133844.5938, 133833.5938, 133833.7656,  ..., 133833.7969,
         133333.3906, 133833.6250]]), tensor([[[    1,     1,     1,  ..., 16950, 16378, 16415],
         [    0,     3,     4,  ..., 16415, 16415, 16378],
         [    2,     0,     3,  ..., 16292, 16950, 16782]]]), tensor([[    2,     1,     0,  1827,  1826, 15442, 15443,     3,  1816,  1817,
             7,     6,     5,    13,    16, 15444, 15445,     8,  1834,  1835,
            15,    14, 14059, 14058, 15446, 15447,  1856,  1858,  1857,    22,
         15465,    32,    31,    30, 13024, 13027, 15448, 15449,    33,  186

#Show data

In [8]:
single_list = os.listdir(single_path)

with h5py.File(single_path + single_list[random.randint(0, len(single_list))] , 'r') as file:

    vertices = np.array(file['pos'])
    faces = np.array(file['face'])

mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.show()

In [9]:
bifurcating_list = os.listdir(bifurcating_path)

with h5py.File(bifurcating_path + bifurcating_list[random.randint(0, len(bifurcating_list))] , 'r') as file:
    vertices = np.array(file['pos'])
    faces = np.array(file['face'])

mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.show()